# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.38,79,58,8.54,TF,1712009565
1,1,alesund,62.4723,6.1549,4.63,82,75,12.35,NO,1712009531
2,2,coracao de jesus,-16.6853,-44.3650,24.08,80,93,1.13,BR,1712009580
3,3,edmundston,47.3737,-68.3251,5.63,38,40,3.09,CA,1712009794
4,4,grytviken,-54.2811,-36.5092,1.49,82,27,4.41,GS,1712009795


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5,
    frame_width = 800,
    frame_height = 500
)   

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) &
                            (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna (how = 'any', axis = 0)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,ad dabbah,18.0500,30.9500,26.03,18,0,3.82,SD,1712009849
53,53,hengchun,22.0042,120.7439,24.34,81,0,3.80,TW,1712009859
89,89,los altos,37.3852,-122.1141,23.65,38,0,3.09,US,1712009735
147,147,nyaunglebin,17.9500,96.7333,24.57,71,0,1.90,MM,1712009981
214,214,al hufuf,25.3646,49.5653,22.19,40,0,1.54,SA,1712010064
239,239,newman,37.3138,-121.0208,21.75,48,0,4.02,US,1712010096
318,318,al burayqah,30.4062,19.5739,22.26,49,0,4.28,LY,1712010202
342,342,kuraymah,18.5500,31.8500,26.70,16,0,4.44,SD,1712010235
408,408,mhamid,29.8200,-5.7200,22.61,22,0,3.06,MA,1712010321
410,410,laguna,38.4210,-121.4238,22.99,35,0,3.60,US,1712010305


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
46,ad dabbah,SD,18.0500,30.9500,18,
53,hengchun,TW,22.0042,120.7439,81,
89,los altos,US,37.3852,-122.1141,38,
147,nyaunglebin,MM,17.9500,96.7333,71,
214,al hufuf,SA,25.3646,49.5653,40,
239,newman,US,37.3138,-121.0208,48,
318,al burayqah,LY,30.4062,19.5739,49,
342,kuraymah,SD,18.5500,31.8500,16,
408,mhamid,MA,29.8200,-5.7200,22,
410,laguna,US,38.4210,-121.4238,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ad dabbah - nearest hotel: No hotel found
hengchun - nearest hotel: 海的顏色精品旅館
los altos - nearest hotel: Mountain View Inn
nyaunglebin - nearest hotel: စန်းဋ္ဌေး
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
newman - nearest hotel: No hotel found
al burayqah - nearest hotel: فندق ماس ليبيا
kuraymah - nearest hotel: Hotel al Nasser
mhamid - nearest hotel: Hotel Kasbah Azalay
laguna - nearest hotel: Holiday Inn Express & Suites
salalah - nearest hotel: Muscat International Hotel
awjilah - nearest hotel: No hotel found
fiambala - nearest hotel: Casona del Pino
brak - nearest hotel: فندق براك السياحي


,City,Country,Lat,Lng,Humidity,Hotel Name
46,ad dabbah,SD,18.0500,30.9500,18,No hotel found
53,hengchun,TW,22.0042,120.7439,81,海的顏色精品旅館
89,los altos,US,37.3852,-122.1141,38,Mountain View Inn
147,nyaunglebin,MM,17.9500,96.7333,71,စန်းဋ္ဌေး
214,al hufuf,SA,25.3646,49.5653,40,Al Muhaidab Residence Al Ahsa
239,newman,US,37.3138,-121.0208,48,No hotel found
318,al burayqah,LY,30.4062,19.5739,49,فندق ماس ليبيا
342,kuraymah,SD,18.5500,31.8500,16,Hotel al Nasser
408,mhamid,MA,29.8200,-5.7200,22,Hotel Kasbah Azalay
410,laguna,US,38.4210,-121.4238,35,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    hover_cols = ['Lng', 'Lat', 'City', 'Humidity', 'Hotel Name', 'Country'],
    frame_width = 800,
    frame_height = 500
)   

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)